In [71]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from util import ReplayBuffer
from util import stats

pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 100)

ImportError: cannot import name 'ReplayBuffer' from 'util' (D:\workspace\python\Research-on-Incremental-Models-for-Real-time-Software-Defect-Prediction\util.py)

In [2]:
df = pd.read_csv(r'.\jit\input\bugzilla.csv')
df['commitdate'] = pd.to_datetime(df['commitdate'])
df['commitdate'] = (df['commitdate'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')

In [45]:
print(df.head())

   transactionid  commitdate  ns  nm  nf   entropy        la        ld          lt  fix  ndev     pd       npt  exp    rexp  sexp  bug
0              3       11668   1   1   3  0.579380  0.093620  0.000000  480.666667    1    14    596  0.666667  143  133.50   129    1
1              7       10876   1   1   1  0.000000  0.000000  0.000000  398.000000    1     1      0  1.000000  140  140.00   137    1
2              8       11822   3   3  52  0.739279  0.183477  0.208913  283.519231    0    23  15836  0.750000  984  818.65   978    0
3              9       11708   1   1   8  0.685328  0.016039  0.012880  514.375000    1    21   1281  1.000000  579  479.25   550    0
4             10       11675   2   2  38  0.769776  0.091829  0.072746  366.815789    1    21   6565  0.763158  413  313.25   405    0


In [50]:
x = df.iloc[:, :-1]
y = df['bug']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)

In [63]:
# Initialize the logistic regression model and the replay buffer
model = LogisticRegression(penalty="l2", C=0.1, max_iter=800)
rb = ReplayBuffer(capacity=100)

# Training loop with memory replay
epochs = 10
batch_size = 32

for epoch in range(epochs):
    for i, (index, row) in enumerate(x_train.iterrows()):
        rb.add((row.values, y_train.iloc[i]))

        if i % batch_size == 0 and len(rb.buffer) >= batch_size:
            # Sample a batch from the buffer
            experiences = rb.sample(batch_size)
            batch_X = np.array([exp[0] for exp in experiences])
            batch_y = np.array([exp[1] for exp in experiences])

            # Train the model on the batch
            model.partial_fit(batch_X, batch_y, classes=np.unique(y))

# Evaluate the model
y_pred = model.predict(x_test)

pydev debugger: Unable to find real location for: C:\Users\Administrator\AppData\Local\Temp\ipykernel_2356\2556658210.py
pydev debugger: Unable to find real location for: C:\Users\Administrator\AppData\Local\Temp\ipykernel_2356\3430571703.py
pydev debugger: Unable to find real location for: <__array_function__ internals>


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [62]:
stats(y_pred, y_test)

NameError: name 'y_pred' is not defined